<a href="https://colab.research.google.com/github/cleopollinger/sql-project/blob/main/ticketmaster_API_Extract_Load_Raw.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
import requests
import json
import pandas as pd


In [13]:
API_KEY = 'ysQXsnMwjDwUmJgIbVNJqqwaOMWWBG9Z'
BASE_URL = 'https://app.ticketmaster.com/discovery/v2/events.json'

# Example parameters
params = {
    'apikey': API_KEY,
    'keyword': 'music',
    'city': 'Los Angeles',
    'size': 200  # max 200 per page
}

response = requests.get(BASE_URL, params=params)

# Check the response
print(response.status_code)  # Should be 200
data = response.json()


200


In [14]:
# Check what keys exist
events = data.get('_embedded', {}).get('events', [])

# Flatten some key info into a list of dicts
event_list = []

for event in events:
    event_info = {
        'artist': event.get('name'),
        'date': event.get('dates', {}).get('start', {}).get('localDate'),
        'time': event.get('dates', {}).get('start', {}).get('localTime'),
        'venue': event.get('_embedded', {}).get('venues', [{}])[0].get('name'),
        'city': event.get('_embedded', {}).get('venues', [{}])[0].get('city', {}).get('name'),
        'country': event.get('_embedded', {}).get('venues', [{}])[0].get('country', {}).get('name'),
    }
    event_list.append(event_info)

# Convert to DataFrame
df = pd.DataFrame(event_list)
df.head()


,artist,date,time,venue,city,country
0,THE SOUND OF MUSIC,2026-05-05,19:30:00,Hollywood Pantages,Los Angeles,United States Of America
1,THE SOUND OF MUSIC,2026-05-06,19:30:00,Hollywood Pantages,Los Angeles,United States Of America
2,THE SOUND OF MUSIC,2026-05-07,19:30:00,Hollywood Pantages,Los Angeles,United States Of America
3,THE SOUND OF MUSIC,2026-05-08,20:00:00,Hollywood Pantages,Los Angeles,United States Of America
4,THE SOUND OF MUSIC,2026-05-09,20:00:00,Hollywood Pantages,Los Angeles,United States Of America


In [16]:
with open('raw_ticketmaster_data.json', 'w') as f:
    json.dump(data, f, indent=4)


In [17]:
df.to_csv('ticketmaster_events.csv', index=False)


In [15]:
from google.colab import sheets
sheet = sheets.InteractiveSheet(df=df)

https://docs.google.com/spreadsheets/d/1VM9s7frg5k3yRQ2NqUgoMFVfN3jFGtmUbdeKCjISgpk/edit#gid=0
